# Transformée en Ondelettes 2D, application au traitement des images

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests
import boto3
import os

In [ ]:
s3_endpoint_url = 'https://object-rook-ceph.apps.math.cnrs.fr/'
s3_access_key_id = '9F7EB8YBUWXDV7A4IZYW' # le contenu de secrets/dossal
s3_secret_access_key = 'skV01Eei5M3xVOxROIDr3qymYhWtkrxPpMyj8nwb' # le contenu de secrets/dossal
s3_bucket = 'signal-image'
s3 = boto3.client('s3',
                  '',
                  endpoint_url = s3_endpoint_url,
                  aws_access_key_id = s3_access_key_id,
                  aws_secret_access_key = s3_secret_access_key)
Data=["Lenna.jpg","Canaletto.jpeg","MinotaureBruite.jpeg","Cartoon.jpg"]
if not os.path.isfile('Lenna.jpg'):
    for filenames in Data:  
        s3.download_file(s3_bucket,filenames,filenames)
def chargeData(name):
    if name=='Lenna':
        res=np.array(Image.open("Lenna.jpg")).astype(float)
    if name=='Canaletto':
        res=np.array(Image.open("Canaletto.jpeg")).astype(float)
    if name=='Minotaure':
        res=np.array(Image.open("MinotaureBruite.jpeg")).astype(float)  
    if name=='Cartoon':
        res=np.array(Image.open("Cartoon.jpg")).astype(float) 
    return res
options1=dict(width=400,height=400,xaxis=None,yaxis=None,toolbar=None)
options2=dict(width=700,height=400,xaxis=None,yaxis=None,toolbar=None)

## Approximation linéaire et non linéaire.

In [ ]:
im2=chargeData('Lenna')
im=chargeData('Canaletto')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options),hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
size=400
WT= pywt.wavedecn(im, 'haar', mode='per', level=2)
arr, coeff_slices = pywt.coeffs_to_array(WT)
hv.Image(arr).opts(cmap='gray',width=size,height=size)

Ecrire une fonction qui réalise une approxiamtion non linéaire en seuillant les coefficients d'ondelettes.
On pourra utiliser les fonctions suivante : pywt.coeffs_to_array et pywt.array_to_coeffs

In [ ]:
def SeuillageOndelette(SB,qmf,L,Seuil):
        WTB= pywt.wavedecn(SB, qmf, mode='per', level=L) #on décompose le signal en coeff
        arr, coeff_slices = pywt.coeffs_to_array(WTB) #on place les coeff dans un tabl
        
        WTS = arr*(np.abs(arr)>Seuil) + Seuil *(np.abs(arr)<Seuil) #on impose une condition sur un seuil choisi
        ncoeffs=len(WTS)
        coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices) # on ne garde que les coeff sup à ce seuil
        Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per') #on recompose le signal avec seulement les coeff sélectionnés
        return Srec,ncoeffs


def ApproxOnd2D(S, qmf, L, threshold): 
    rows, cols = S.shape
    Lmax_rows = pywt.dwt_max_level(rows, pywt.Wavelet(qmf).dec_len) #calcul du niveau maximal de décomposition du signal possible
    L1 = min(L, Lmax_rows) #pour ne pas avoir un trop gros niveau de décomposition on prend le min entre Lmax ci-dessus et un L arbitrairement choisi
    Srec,ncoeffs=SeuillageOndelette(S,qmf,L1,threshold) #on applique le seuillage dur avec le niveau de décomposition choisi au-dessus
    return Srec, ncoeffs

Le seuillage effectué ici est appelé seuillage doux. Il affecte la valeur d'un seuil à tous les pixels de l'image en dessous de ce seuil. Cela permet d'avoir une reconstruction assez 'continue' du signal tout en gardant les valeurs des pixels les plus importantes !

In [ ]:
s=10
#s=25
#s=50
L=6
Irec,p=ApproxOnd2D(im,'db2',L,s)
#On remarque qu'à Seuil=10 cela marche très bien
#Cependant si le Seuil est trop grand (par exemple plus de 50) on a beaucoup de pixel valant 0 pour la reconstruction
hv.Image(Irec).opts(cmap='gray',width=400,height=400)

Ecrire une focntion PSNR.

In [ ]:
def PSNR(I,Iref):
    mse = np.mean( (Iref - I) ** 2 ) #on calcule l'erreur moyenne entre une image de référence et une autre image (souvent reconstruite ou bruitée) 
    if mse == 0: #si c'est la même image on renvoit PSNR = 100
        return 100
    Val_MAX = np.max(Iref) #sinon on récupérer le max de Iref et on applique la formule du PSNR
    return 20 * np.log10(Val_MAX / np.sqrt(mse))

Ecrire une fonction qui réalise une approximation non linéaire en conservant un nombre N de coefficients d'ondelettes et la tester. On pourra utiliser les fonctions pywt.ravel_coeffs et unravel_coeffs.

In [ ]:
def ApproxOnd2nonlin(I,qmf,L,N):
    #on fait de même que pour le seuillage dur
    rows, cols = I.shape
    Lmax_rows = pywt.dwt_max_level(rows, pywt.Wavelet(qmf).dec_len)
    L1 = min(L,Lmax_rows)
    WT = pywt.wavedecn(I, qmf, mode='per', level=L1)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WT)
    
    Ind = np.argsort(np.abs(arr)) #on trie les valeurs absolues par ordre croissant et on récupère les indices ordonnés dans Ind
    
    WTS = np.zeros_like(arr)
    WTS[Ind[-N:]] = arr[Ind[-N:]]  #on garde les N coeffs les plus grands
    
    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices, coeff_shapes)
    Irec = pywt.waverecn(coeffs_from_arr, qmf, mode='per')
    
    p = PSNR(Irec, I) #on vérifie la quamlité de la reconstruction de l'image en affichant le PSNR
    return Irec, p

In [ ]:
N=20000 #le nombre de coeffs à garder plus il est grand mieux c'est !
Irec,p=ApproxOnd2nonlin(im,'db2',L,N)
hv.Image(Irec).opts(title=f'Image reconstruite, PSNR={p:.2f}', cmap='gray',width=400,height=400)

Créer un Dashboard qui permet d'explorer la fonction précédente.

In [ ]:
# Créer la classe Approx2D pour mieux visualiser l'impact des différents paramètres sur notre fonction
wavelist = ['db2','db3','db4','coif1','coif2','coif3']

class Approx2D(param.Parameterized):
    #on crée les références de chaque paramètres qu'on veut utiliser
    #on définit à chaque fois la valeur de défaut du paramètre et les différentes modalités/valeurs qu'il peut prendre dans notre dashboard
    Image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    qmf = param.ObjectSelector(default="db2", objects=wavelist)
    L = param.Integer(5, bounds=(0,7))
    N = param.Integer(2000, bounds=(1, 10000))

    def view(self):
        I=imagesRef[self.Image]

        Irec, p = ApproxOnd2nonlin(I, self.qmf, self.L, self.N)

        return pn.Row(hv.Image(I).opts(title=f'Image de référence',cmap='gray'),
                      hv.Image(Irec).opts(title=f'Image reconstruite, PSNR={p:.2f}', cmap='gray')) 
    #pour afficher l'image reconstruite et le PSNR par rapport à l'image de référence

In [ ]:
approx2D = Approx2D()
pn.Row(approx2D.param,approx2D.view) #pour afficher le dashboard

**Interprétation :** 
On remarque que plus on garde de coeffs meilleur est la reconstruction. De plus si on met un L, donc le niveau de décomposition, trop faible (c'est à dire inférieur à 3) notre reconstruction est également mauvaise.

Créer un plan d'experiences qui permet d'explorer la fonction ApproxOnd2nonlin

In [ ]:
#On crée un plan d'expériences pour ne pas à avoir à recompiler notre fonction "ApproxOnd2nonlin" à chaque modification de paramètres
#Ainsi on met dans un tableau différentes valeurs de paramètres à tester et on y applique la fonction "ApproxOnd2nonlin"
import itertools
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences = {'Image':imagesRef,'N':np.linspace(1000,50000,40, dtype='int'), 'level' : range(1,7), 'wave':wavelist}
dfexp=pd.DataFrame(list(itertools.product(*experiences.values())),columns=experiences.keys())
#on applique la fonction et on récupère le résultat dans une nouvelle colonne
dfexp['Result'] = dfexp.apply(lambda row : ApproxOnd2nonlin(imagesRef[row["Image"]],row["wave"],row["level"],row["N"]),axis=1)
print(dfexp)

Créer la fonction qui à une ligne de la base de donnée précédente calcule le PSNR associé.

In [ ]:
def row2PSNR(row): #on crée une fonction pour récupérer le PSNR dans la colonne résultat de chaque line
    return row["Result"][1]

Appliquer la fonction sur la base de donnée et ajouter la colonne PSNR à la base de données dfexp 

In [ ]:
dfexp['PSNR'] = dfexp.apply(lambda row : row2PSNR(row), axis = 1)
print(dfexp)

Utiliser hvplot pour visualiser la base de données.

In [ ]:
import hvplot.pandas

In [ ]:
from bokeh.models import HoverTool
h = HoverTool()

In [ ]:
#on affiche le PSNR en fonction du nombre de coefficients qu'on garde pour chaque base d'ondelettes.
dfexp.hvplot('N','PSNR', by='wave',kind='scatter',groupby=['Image','level']).opts(width=600,tools=[h]).redim.range(PSNR=(0,80),N=(0,50000))

On peut donc chercher la base d'ondelettes la plus optimale, c'est à dire celle qui a le PSNR le plus grand en fonction de nombre de coeffs N qu'on veut garder et du niveau de décomposition.

On remarque que pour L=3 c'est coif3 la meilleure base d'ondelettes quelque soit le nombre de coefficients gardés 

## Débruitage d'images

Ecrire une fonction qui effectue un seuillage dur en ondelettes et la tester. On pourra utiliser la fonction pywt.ravel_coeffs et on pensera à cliper le résultat entre 0 et 255.

In [ ]:
import pywt
import numpy as np

def SeuillageDurOndelettes(S,qmf,L,Seuil): #la différence avec le doux : si <Seuil => on met 0 au lieu de la valeur du Seuil
    rows, cols = S.shape
    Lmax_rows = pywt.dwt_max_level(rows, pywt.Wavelet(qmf).dec_len)
    L1 = min(L, Lmax_rows)
    WTB= pywt.wavedecn(S, qmf, mode='per', level=L) #on décompose le signal en coeff
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WTB)
    WTS = arr*(np.abs(arr)>Seuil) #on impose une condition sur un seuil choisi
    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices, coeff_shapes)
    Srec=pywt.waverecn(coeffs_from_arr,qmf,mode='per') #on recompose le signal avec seulement les coeff sélectionnés    
    return Srec


Le seuillage effectué ici est un seuillage dur. Il met à 0 tous les coefficients qui dépassent un certain seuil. Cela permet de conserver l'amplitude du signal mais la reconstruiction est plus 'discontinue' qu'avec un seuillage doux

Construire un dashboard qui permet d'explorer la fonction SeuillageDurOndelettes.

In [ ]:
SeuillageDurOndelettes(imagesRef['Lenna'], 'haar', 3, 5)

In [ ]:
class WaveSeuillage(param.Parameterized): #de même que précédemment on crée un dashboard pour tester l'impact de nos paramètres sur notre fonction de Seuillage Dur
    Image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys())
    qmf = param.ObjectSelector(default="haar",objects=wavelist)
    Seuil = param.Number(10,bounds=(1,1000))
    L = param.Integer(5, bounds=(0,7))
    def view(self):
        I=imagesRef[self.Image]
        Irec=SeuillageDurOndelettes(I,self.qmf,self.L,self.Seuil)
        return pn.Row(hv.Image(I).opts(title=f'Image de référence',cmap='gray'), #pour afficher l'image de référence à côté de l'image reconstruite
                      hv.Image(Irec).opts(title=f'Image reconstruite',cmap='gray')) #pour afficher l'image reconstruite

In [ ]:
waveseuillage = WaveSeuillage()
pn.Row(waveseuillage.param, waveseuillage.view)

On remarque que plus le seuil est elevé moins l'image est bien reconstruite, en effet on perds plus de coefficients, dont certains qui sont importants pour l'image, alors qu'avec un seuil petit on garde tous les coefficients important pour cette dernière !

In [ ]:
#On veut désormais tester nos fonctions avec une image bruitée, donc on crée cette dernière :
n1,n2=np.shape(im)
B=np.random.randn(n1,n2) # le coeff de bruitage qui est un nombre aléatoire de la taille de votre image
sigma=100 # pour augmenter la valeur des bruits
ib=im+sigma*B #pour créer l'image bruitée
ib=np.clip(ib,0,255)
hv.Image(ib).opts(cmap='gray',width=400,height=400) #pour visualiser l'image qu'on vient de bruiter

Ecrire un dashboard qui permet de visualiser rapidement l'effet d'un débruitage en ondelettes et qui renvoie les images originales, bruitées et débruitées ainsi que les PSNR associés aux images bruitéeset débruitées.

In [ ]:
class WaveDebruit(param.Parameterized): #on crée un dashboard pour voir comment se comporte notre fonction en présence d'une image bruitée
    image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys())
    qmf = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7)) #niveau de décomposition dans la composition de Haar
    Seuil = param.Number(3,bounds=(1,500)) #valeur de seuil pour les fonctions de seuillage
    Sigma = param.Number(10,bounds=(1,100)) #pour modifier l'importance des bruits (plus sigma est grand plus les bruits ont une grande valeur)
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        np.random.seed(self.seednoise)
        I=imagesRef[self.image]
        n1,n2=np.shape(I)
        B=np.random.randn(n1,n2)
        ib=I+self.Sigma*B #pour créer l'image bruitée tout en pouvant modifier le bruit avec le sigma
        ib=np.clip(ib,0,255)
        
        Irec=SeuillageDurOndelettes(ib,self.qmf,self.L,self.Seuil)
        #on calcule les PSNRs
        PSNR_bruit = PSNR(ib,I)
        PSNR_seuil = PSNR(Irec,I)
        return pn.Row(hv.Image(ib).opts(title=f'Image bruitée, PSNR={PSNR_bruit:.2f}',cmap='gray'), #on affiche l'image bruitée ainsi que son PSNR avec l'image originelle
                      hv.Image(Irec).opts(title=f'Image reconstruite, PSNR={PSNR_seuil:.2f}', cmap='gray')) #on affiche l'image reconstruite ainsi que son PSNR avec l'image originelle

In [ ]:
wavedebruit = WaveDebruit()
pn.Row(wavedebruit.param,wavedebruit.view)

On remarque donc que plus le bruit est élevé, ainsi plus sigma est grand, donc les valeurs du bruit sont grandes, moins notre fonction de seuillage fonctionne. En effet, elle a du mal à faire la différence entre les coefficients importants pour la reconstruction de l'image et le strop grandes valeurs de bruit. ALors que pour des plus petits bruits elle se débrouille mieux, les coefficients de l'image étant plus élevés que le bruit.

## Débruitage d'images et translations

Ecrire une fonction qui réalise un débruitage avec une moyenne sur des NbT fois NbT translations et la tester. Vérifier le gain en PNSR.

In [ ]:
def DebruitTranslation(IB,wave,seuil,NbT): #on inclut la notion de translation
    Lmax=pywt.dwt_max_level(len(IB),pywt.Wavelet(wave).dec_len)
    ISum=0*IB
    P=np.zeros(NbT)
    for k in np.arange(0,NbT):
        IBtemp=np.roll(IB,k) #roll permet de décaler de k vers la droite les éléments d'un vecteur 
        Irectemp=SeuillageDurOndelettes(IBtemp,wave,Lmax,seuil) #base d'ondelette sur le signal translaté de k
        #la transformée en ondelettes est variante par translation don ça nous permet de recup + d'infos
        Irectemp2=np.roll(Irectemp,-k) #signal reconstruit de la base d'ondelettes redécalé de k dans l'autre sens 
        ISum=ISum+Irectemp2
        Irec=ISum/(k+1)
        P[k]=PSNR(IB,Irec)
    return Irec #,P

Créer un dasboard pour explorer la fonction précédente. La sortie doit aussi être composée de 3 images et 2 PSNR.

In [ ]:
class Debruit_translat(param.Parameterized): #de même que pour les dahsboards prédécents.
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    NbT = param.Integer(2,bounds=(1,20)) #pour faire varier le nombre de translations que l'on veut faire 
    Sigma = param.Number(10,bounds=(1,100))
    seednoise = param.Integer(1,bounds=(0,50))
    seuil=param.Integer(10,bounds=(0,500))
    def view(self):
        np.random.seed(self.seednoise)
        I=imagesRef[self.image]
        n1,n2=np.shape(I)
        B=np.random.randn(n1,n2)
        ib=I+self.Sigma*B
        ib=np.clip(ib,0,255)
        Irec=DebruitTranslation(ib,self.wave,self.seuil,self.NbT)
        #on calcule nos PSNRs
        PSNR_bruit = PSNR(ib,I)
        PSNR_seuil = PSNR(Irec,I)
        return pn.Row(hv.Image(ib).opts(title=f'Image bruitée, PSNR={PSNR_bruit:.2f}',cmap='gray'), hv.Image(Irec).opts(title=f'Image reconstruite, PSNR={PSNR_seuil:.2f}', cmap='gray'), hv.Image(I).opts(title=f'Image', cmap='gray'))

In [ ]:
wavedebruit_trans = Debruit_translat()
pn.Row(wavedebruit_trans.param,wavedebruit_trans.view)

On remarque que plus l'on effectue de translations, ainsi plus Nbt est grand, mieux notre image est reconstruite. Cependant, on remarque qu'une faible variation positive du PSNR à partir de Nbt = 10 environ. Sinon on a les mêmes interprétations au niveau du seuil que précédemment.

## Débruitage d'une image couleur.

Pour effectuer le débruitage d'une image générale, c'est à dire d'une image couleur dont le format n'est pas carré et dont les dimensions ne sont pas des puissances de 2 on procède comme suit :

1) On effectue un débruitage séparé sur chacun des canaux.

2) Le format carré n'est pas un vrai problème, il faut juste que les dimensions soient des multiples de puissances de 2. C'est la puissance de 2 qui définira l'échelle maximale de la décomposition en ondelettes. Il est donc préférable que les dimensions de l'images soient un petit multiple d'une puissance de 2.

3) On étend l'image par symétrie ou périodicité pour qu'elle ait les dimensions souhaitées. A la fin du processus de débruitage on tronque le résultat obtenu à la dimension de l'image originale.

4) Si le niveau de bruit n'est pas connu, il faut l'estimer en utilisant les coefficients d'ondelettes de la plus petite échelle (voir le notebook sur le débruitage de signaux).


# Proposer une fonction qui effectue le débruitage d'une image couleur de dimensions quelconques. 

La fonction peut prendre en entrée un tableau numpy ou une image dans une format d'images classique.
Vous pouvez tester votre programme en bruitant vous même une ou plusieurs images de référence et évaluer le gain en terme de PSNR.

In [ ]:
#1- On débruite pour chaque canaux donc chaque couleur du spectre RGB
#Avant on avait une mat 256*256 avec une valeur de nuance de gris
#maintenant on a une mat 256*256*3 car il y a des nuances de Red Green Blue
def Debruit_Couleur(IB,wave,seuil,NbT):
    IB_r, IB_g, IB_b =np.split(IB,3,axis=2) #ici on sépare donc en fonction des coeffs r g et b
    #on obtient 3 matrices dont chaque élément est un tableau comportant un élément 
    #au niveau des dimensions on passe de 256*256*3 <=> 256*256*1
    #pour utiliser notre fonction de Débruitage il nous faut recup le coeff de ces tableaux
    IB_r=IB_r[:,:,0];IB_g=IB_g[:,:,0];IB_b=IB_b[:,:,0]
    
    #on applique notre fonction DebruitTranslation pour chaque couleur RGB
    Irec_r=DebruitTranslation(IB_r,wave,seuil,NbT)
    Irec_g=DebruitTranslation(IB_g,wave,seuil,NbT)
    Irec_b=DebruitTranslation(IB_b,wave,seuil,NbT)
    Irec = np.stack([Irec_r, Irec_g, Irec_b],axis=2) #on réassemble nos résultats après les avoir calculés séparémment 
    return Irec


## Pour aller plus loin (à titre informatif et optionnel)

On peut améliorer les méthodes par seuillage dans une base d'ondelettes en effectuant un seuillage par blocs. C'est à dire, ne pas décider de conserver ou pas un coefficients en fonction de sa seule amplitude mais plutôt en fonction de l'énergie d'un voisinage de coefficients. 

Voir : http://www.cnrs.fr/insmi/spip.php?article265

En effet, il est rare qu'un coefficient soit significatif seul au milieu de coefficients nuls. 

La mméthode de sueillage par blocs consiste à choisir une taille de voisinage (par exemple 4*4 coeffients en dimension 2) pour une échelle et une direction donnée et de conserver l'intégralité des coefficients si l'énergie (la somme des carrés des coefficients) est supérieure à un seuil et de les mettre tous à 0 si ce n'est pas le cas. 

Dans ce cas aussi, les translations permettent d'améliorer le rendu visuel en limitant les effets de blocs.

On peut aussi constuire des blocs "3D" en considérant des blocs qui comprennent les coefficients des 3 créneaux de couleurs. L'idée est de corréler le débruitage un peu à travers l'espace et l'espace des couleurs.

Il est possible d'effectuer un débruitage en changeant d'espace colorimétrique en passant du RGB au YUV par exemple.

# Débruiter un minotaure ?

A l'aide de tout ce qui a été fait précédemment, proposer une version débruitée de l'image couleur contenue dans le tableau Mi

In [ ]:
Mi=chargeData('Minotaure')
Minotaure=np.clip(Mi,0,255)
hv.RGB(Minotaure.astype('uint8')).opts(xlabel=None,ylabel=None,width=400,height=500)

Rédiger également une fonction prenant en entrée un nom de fichier permettant de calculer le PSNR de votre proposition d'image débruitée avec l'image en question. On calcule le PSNR entre deux images couleurs en calculant la somme des erreurs quadratiques sur les 3 canaux.

Attention, l'image a 3 canaux de couleur, n'est pas carrée et les dimensions ne sont pas des puissances de 2.

In [ ]:
def PSNR_couleur(I,Iref): #on doit donc modifier notre fonction PSNR couleur car on doit calculer le PSNR pour chaque couleur RGB 
    #Comme précédemment on sépare les couleur : 
    I_r, I_g, I_b =np.split(I,3,axis=2) ; I_r=I_r[:,:,0] ; I_g=I_g[:,:,0] ; I_b=I_b[:,:,0]
    Iref_r, Iref_g, Iref_b =np.split(Iref,3,axis=2) ; Iref_r=Iref_r[:,:,0] ; Iref_g=Iref_g[:,:,0] ; Iref_b=Iref_b[:,:,0]
    #Ensuite on reprend notre calcul du PSNR classique
    mse_r = np.mean( (Iref_r - I_r) ** 2 )
    mse_g = np.mean( (Iref_g - I_g) ** 2 )
    mse_b = np.mean( (Iref_b - I_b) ** 2 )
    #Puis on somme
    mse=mse_r + mse_g + mse_b
    if mse == 0:
        return 100
    Val_MAX = np.max(Iref)
    return 20 * np.log10(Val_MAX / np.sqrt(mse))

In [ ]:
Ca=chargeData('Cartoon')
imagesRefCol= {"Minotaure" : Mi, "Cartoon" : Ca}
class Debruit_translat_couleur(param.Parameterized):
    #image = param.ObjectSelector(default="Minotaure",objects=imagesRefCol.keys())
    image=Minotaure
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    NbT = param.Integer(2,bounds=(1,10)) #on se limite à 10 car précédemment on n'a pas vu d'évolutions pour Nbt > 10
    #on n'a plus besoin de bruit car l'image est déjà bruitée d'origine
        #Sigma = param.Number(10,bounds=(1,30))
        #seednoise = param.Integer(1,bounds=(0,50))
    seuil=param.Integer(10,bounds=(0,100))
    def view(self):
        #on n'a plus besoin de bruit car l'image est déjà bruitée d'origine
            #np.random.seed(self.seednoise)
            #I=imagesRef[self.image]
            #n1,n2=np.shape(I)
            #B=np.random.randn(n1,n2)
            #ib=I+self.Sigma*B
            #ib=np.clip(ib,0,255)
        Irec=Debruit_Couleur(self.image,self.wave,self.seuil,self.NbT)
        #On calcule le PSNR avec l'IMage Bruitée donc attention l'interprétation change !!
        P=PSNR_couleur(self.image,Irec)
        
        return pn.Row(hv.RGB(self.image.astype('uint8')).opts(title=f'Image bruitée'), 
                      hv.RGB(Irec.astype('uint8')).opts(title=f'Image reconstruite, PSNR={P:.2f}'))

In [ ]:
wavedebruit_trans_couleur = Debruit_translat_couleur()
pn.Row(wavedebruit_trans_couleur.param,wavedebruit_trans_couleur.view)

Attention ici le PSNR est à interpréter différemment. En effet, on n'a, ici, pas d'image de référence donc on ne peut pas calcule rle PSNR avec cette dernière, donc on le calcule avec l'Image bruitée à droite. Or comme le PSNR nous indique la similarité entre deux images, donc ici il est proche de 0, meilleure est la reconstruction et le débruitage.

Ici, nous ne remarquons pas d'influence signifcative de la variable Nbt ainsi le nombre de translations. Cela viendrait probablement du fait que l'image est "peu" bruitée, donc avec des petits coefficients de bruitage, ainsi les translations ne sont pas nécessaires et ont peu d'impact sur la qualité de la reconstruction.

### Plan d'expériences pour évaluer l'impact des translations  

Créer un plan d'expériences pour explorer les performances de l'invariance par translation pour le débruitage. 

In [ ]:
def Bruitage(I,Sigma): #on crée une fonction qui bruite directement nos images avec la même méthode que précédemment
    n1,n2=np.shape(I)
    B=np.random.randn(n1,n2)
    ib=I+Sigma*B
    ib=np.clip(ib,0,255)
    return ib

In [ ]:
#on fait un plan d'expériences, c'est à dire qu'au lieu d'un dahsboard qui recompile notre fonction à chaque fois, 
#on la compile directement pour pleins de valeurs différentes pour afficher nos résultats
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences_DebruitTrans = {'Image': imagesRef.keys(),
                            'NbT':np.arange(1,10),
                            'wave':wavelist,
                            'Seuil':np.linspace(50,150,2),
                            'Sigma':np.linspace(10,100,10)}
dfexp_DebruitTrans=pd.DataFrame(list(itertools.product(*experiences_DebruitTrans.values())),columns=experiences_DebruitTrans.keys()) #on crée donc un dataframe
#ce dataframe contient sur chaque ligne tous nos paramètres qu'on utilisera avec notre fonction DebruitTranslation ainsi chaque ligne correspond à une configuration possible.
dfexp_DebruitTrans['Result'] = dfexp_DebruitTrans.apply(lambda row : DebruitTranslation(imagesRef[row["Image"]],row["wave"],row["Seuil"],row["NbT"]),axis=1) #on applique notre fonction DebruitTranslation à chaque ligne

In [ ]:
print(dfexp_DebruitTrans) #pour afficher notre plan d'exp

Ecrire une fonction qui calcule le PSNR moyen sur n réalisations de bruit du débruitage d'une image avec NbT*NbT translations (qui utilise par exemple la fonction DebruitTranslation)

In [ ]:
def Debruit_Translat_PSNRMoyen(I,wave,sigma,seuil,NbT,n): #on effectue plusieurs translations et on calcule le psnr moyen
    psnr=0
    for i in range(n):
        ib = Bruitage(I,sigma)
        Irec=DebruitTranslation(ib,wave,seuil,NbT)[0]
        psnr+=PSNR(I,Irec)/n
    return psnr

Ecrire la fonction qui à une ligne de la base de données précédente calcule le PSNR moyen sur 4 réalisations du bruit. Puis l'appliquer à la base de données et ajouter la colonne des PSNR calculés à la base de données.

In [ ]:
def row2DebruitTrans(row): #on fait une fonction à qui on donnera une ligne et qui calculera le psnr moyen avec notre fonction ci-dessu.
    return Debruit_Translat_PSNRMoyen(imagesRef[row['Image']],
                                      row['wave'],
                                      row['Sigma'],
                                      row['Seuil'],
                                      row['NbT']
                                      ,4)

In [ ]:
dfexp_DebruitTrans['PSNR Moyen'] = dfexp_DebruitTrans.apply(lambda row : row2DebruitTrans(row), axis=1)

Utiliser hvplot pour visualiser les résulatst contenus dans la base de données.

In [ ]:
#On affiche le graphe du PSNR moyen en fonction des nombres de translation et des différentes bases d'ondelettes, tout en pouvant modifier sigma, donc le bruit, et le seuil.

dfexp_DebruitTrans.hvplot('NbT','PSNR Moyen', by='wave',kind='scatter',groupby=['Image','Sigma', 'Seuil']).opts(width=600,tools=[h]).redim.range(PSNR=(0,80),N=(0,5))

On retrouve qu'un petit nombre de translations diminue légèrement le PSNR donc la qualité de reconstruction de l'image, cependant à partir de 4 translations cela commence à remontrer mais cela stagne 

# Quantification et Entropie de Shannon

In [ ]:
def ShannonEntropy(x):
    value,counts = np.unique(x, return_counts=True)
    Proba=counts/len(x)
    Ent=-np.sum(np.log2(Proba)*Proba)
    return Ent

In [ ]:
x=np.array([13,13,2,7,13,7,1,13]) 
print(ShannonEntropy(x))

In [ ]:
y=np.array([-2,-3,1,0,1,0,-2,-3])
print(ShannonEntropy(y))

Ecrire une fonction qui effectue la quantification de la transformée en ondelettes avec un pas "Pas". On pourra à nouveau utiliser la commande pywt.ravel_coeffs. La fonction doit renvoyer l'image calculée par quantification, le PSNR associé ainsi que le nombre d'octets estimé par la valeur de l'entropie a priori nécessaire pour coder une telle image. On considérera qu'on code séparément les coefficients d'échelle et les coefficients d'ondelettes. Tester la fonction.

In [ ]:
def QuantificationOndelettes(I,qmf,Pas):
    Lmax = pywt.dwt_max_level(len(I), pywt.Wavelet(qmf).dec_len)
    W = pywt.wavedecn(I, qmf, mode='per', level=Lmax)
    arr, coeff_slices = pywt.coeffs_to_array(W)
    arr2=arr.copy()
    arr2=np.round(arr2/Pas) * Pas
    fac_compression = ShannonEntropy(arr.ravel()) / ShannonEntropy(arr2.ravel()) #calcul du facteur de compression
                                                                                       # Donc du nouveau nombre d'octets comparé à l'ancien
    coeffs_from_arr = pywt.array_to_coeffs(arr2, coeff_slices)
    Irec = pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    P = PSNR(I,Irec)
    return Irec, P, fac_compression

Créer le dashboard asscoié à la focntion précédente. 
Le dashboard doit renvoyer l'image quantifiée, le PSNR de l'image ainsi que le facteur de compression théorique associé. 

In [ ]:
I = im
Pas = 30 
Irec, psnr, compression = QuantificationOndelettes(I,'haar',Pas)
print("Facteur de compression :", compression)
hv.Image(Irec).opts(title=f'Image reconstruite, PSNR={psnr:.2f}',cmap='gray')

In [ ]:
class WaveQuant(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    QS = param.Number(30,bounds=(10,300))
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    def view(self):
        I=imagesRef[self.image]
        Irec, psnr, compression = QuantificationOndelettes(I,self.wave,self.QS)
                
        return pn.Row(hv.Image(I).opts(title=f'Image,PSNR',cmap='gray'),
                      hv.Image(Irec).opts(title=f'Image reconstruite, PSNR={psnr:.2f}',cmap='gray'))

    

In [ ]:
waveQuant=WaveQuant()
pn.Row(waveQuant.param,waveQuant.view)

Créer dun plan d'expériences pour comparer les différentes ondelettes pour la quantification... et poursuivre jusqu'à obtenir un affichage de la base de données ainsi créée avec hvplot.

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences_WaveQuant = {'Image': imagesRef.keys(),
                         'QS':np.linspace(30,200,10),
                         'wave':wavelist}

dfexp_WaveQuant=pd.DataFrame(list(itertools.product(*experiences_WaveQuant.values())),columns=experiences_WaveQuant.keys())

print(dfexp_WaveQuant)

In [ ]:
def row2PSNR(row):
    return row['Result'][1]

In [ ]:
def row2DistorsionRate(row):
    return row['Result'][2]

In [ ]:
dfexp_WaveQuant['Result'] = dfexp_WaveQuant.apply(lambda row : QuantificationOndelettes(imagesRef[row["Image"]],row["wave"],row["QS"]),axis=1)

In [ ]:
dfexp_WaveQuant['PSNR'] = dfexp_WaveQuant.apply(lambda row : row2PSNR(row), axis = 1)
dfexp_WaveQuant['Compression'] = dfexp_WaveQuant.apply(lambda row : row2DistorsionRate(row), axis = 1)


print(dfexp_WaveQuant)

In [ ]:
dfexp_WaveQuant.hvplot('Compression','PSNR', by='wave',kind='scatter',groupby=['Image','QS']).opts(width=600,tools=[h]).redim.range(PSNR=(0,110),Compression=(0,120))

## Pour aller plus loin (à titre informatif et optionnel)

Nous proposons ici d'effectuer la compression sur les 3 canau RGB. Or l'oeil humain est plus sensible à la luminance qu'aux composantes purement chromatiques. C'est pourquoi, la plupart des algorithmes de compressions sont effectué dans un espace colorimétrique YUV où Y est la luminance. On alloue alors plus d'information au canal Y et on comprime plus drastiquement les deux autres canaux. Une méthode standart consiste par exemple à sous-échantionner d'un facteur 2 les deux composantes U et V avant de les comprimer. 

https://fr.wikipedia.org/wiki/Sous-échantillonnage_de_la_chrominance

On obtient alors des images de chrominances moins résolues et donc moins lourdes mais le rendu final reste correct car l'oeil humain est nettement plus sensible à la luminance. 